## IMPORT LIBRARIES

In [ ]:
# Import necessary libraries
import tkinter as tk
from tkinter import simpledialog, filedialog
import os
import pandas as pd
from pyDOE2 import bbdesign

## CREATE BOX-BEHNKEN DESIGN PLAN

In [ ]:
# Function to generate a DataFrame with factor names
def generate_factor_df(factors):
    """
    Generate a DataFrame containing factor names.

    Args:
    - factors (list): List of factor names.

    Returns:
    - df (DataFrame): DataFrame representing factor names.
    """
    # Create a dictionary to hold factor names
    data = {'Factor': factors}
    # Convert the dictionary into a DataFrame
    df = pd.DataFrame(data)
    return df

# Function to generate a Box-Behnken design DataFrame
def generate_bb_design(factor_df, center_points=0):
    """
    Generate a Box-Behnken design DataFrame based on specified factors and center points.

    Args:
    - factor_df (DataFrame): DataFrame containing factor names.
    - center_points (int): Number of center points to include in the design.

    Returns:
    - df (DataFrame): DataFrame representing the Box-Behnken design.
    """
    # Extract factor names from the DataFrame
    factors = factor_df['Factor'].tolist()

    # Generate the Box-Behnken design using the specified factors and center points
    design = bbdesign(len(factors), center=center_points)

    # Create DataFrame with appropriate column names
    column_names = factors.copy()
    df = pd.DataFrame(design, columns=column_names)
    return df

# Define the custom mapping function
def custom_mapping(value):
    """
    Custom mapping function to transform values according to specific rules.

    Args:
    - value (int): Value representing factor level.

    Returns:
    - mapped_value (float): Transformed value.
    """
    return value

# Function to handle user input
def get_user_input():
    """
    Prompt the user to input factors, number of replicates, Excel file name, and export folder.

    Returns:
    - factors (list): List of factor names.
    - num_replicates (int): Number of replicates.
    - excel_file_name (str): Excel file name.
    - export_folder (str): Export folder path.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Prompt user for the number of factors
    num_factors = simpledialog.askinteger("Input", "Enter the number of factors:")

    factors = []

    # Prompt user to enter factor names
    for i in range(num_factors):
        factor = simpledialog.askstring("Input", f"Enter the name of factor {i+1}:")
        factors.append(factor)

    # Prompt the user for the number of center points
    center_points = simpledialog.askinteger("Input", "Enter the number of center points (default is 3): ", initialvalue=3)
   
    # Prompt user to enter the number of replicates
    num_replicates = simpledialog.askinteger("Input", "Enter the number of replicates:", initialvalue=1)

    # Prompt user to enter the Excel file name
    excel_file_name = simpledialog.askstring("Input", "Enter the Excel file name:")

    # Prompt user to select the export folder
    export_folder = filedialog.askdirectory(title="Select Export Folder")

    return factors, center_points, num_replicates, excel_file_name, export_folder

# Main function
def main():
    try:
        # Get user input
        factors, center_points, num_replicates, excel_file_name, export_folder = get_user_input()

        # Generate factor DataFrame
        factor_df = generate_factor_df(factors)

        # Check if there are at least 3 factors
        if len(factors) < 3:
            raise ValueError("Box-Behnken design requires at least 3 factors.")

        # Create Box-Behnken design DataFrame
        optimization_df = generate_bb_design(factor_df, center_points)

        # Apply custom mapping to each value in the DataFrame (not needed in this case)

        # Duplicate the design DataFrame
        optimization_df_duplicated = pd.concat([optimization_df] * num_replicates, ignore_index=True)

        # Merge the DataFrames
        merged_df = pd.concat([factor_df, pd.DataFrame(columns=['']), optimization_df_duplicated], axis=1)

        # Add a blank column with heading 'Results' after the last factor
        merged_df.insert(len(merged_df.columns), 'Results', '')

        # Append '.xlsx' extension if not provided
        if not excel_file_name.endswith('.xlsx'):
            excel_file_name += '.xlsx'

        # Write the merged DataFrame to an Excel file in the export folder
        excel_file_path = os.path.join(export_folder, excel_file_name)
        merged_df.to_excel(excel_file_path, index=False)
        print("Excel file saved successfully.")

    except Exception as e:
        print("An error occurred:", e)

if __name__ == "__main__":
    main()
